In [1]:
import os
import shutil
import subprocess
import tempfile
import sys

def run_cmd(cmd):
    """Chạy lệnh và in kết quả, không dừng khi lỗi"""
    print(f"\n>>> {cmd}")
    try:
        subprocess.run(cmd, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Lỗi khi chạy: {cmd}\nMã lỗi: {e.returncode}")
        return False
    return True

# 1. Dọn cache pip
run_cmd("pip cache purge")

# 2. Kiểm tra dung lượng trống
tmp_dir = tempfile.gettempdir()
stat = shutil.disk_usage(tmp_dir)
free_gb = stat.free / (1024 ** 3)
print(f"\nDung lượng trống tại {tmp_dir}: {free_gb:.2f} GB")

# 3. Nếu dung lượng < 2GB thì đổi thư mục tạm sang nơi khác
if free_gb < 2:
    new_tmp = os.path.expanduser("~/temp_pip")
    os.makedirs(new_tmp, exist_ok=True)
    os.environ["TMPDIR"] = new_tmp
    print(f"Đặt thư mục tạm mới: {new_tmp}")
else:
    new_tmp = tmp_dir

# 4. Cài spaCy (thử với --no-cache-dir nếu lỗi)
if not run_cmd(f'"{sys.executable}" -m pip install -U spacy'):
    print("Thử lại với --no-cache-dir để tiết kiệm dung lượng...")
    run_cmd(f'"{sys.executable}" -m pip install --no-cache-dir -U spacy')

# 5. Tải model ngôn ngữ nhỏ (tiếng Anh)
run_cmd(f'"{sys.executable}" -m spacy download en_core_web_sm')

print("\nHoàn tất cài đặt spaCy và model ngôn ngữ!")


>>> pip cache purge

Dung lượng trống tại C:\Users\asus\AppData\Local\Temp: 1.80 GB
Đặt thư mục tạm mới: C:\Users\asus/temp_pip

>>> "c:\Users\asus\venv\llamaenv\Scripts\python.exe" -m pip install -U spacy

>>> "c:\Users\asus\venv\llamaenv\Scripts\python.exe" -m spacy download en_core_web_sm

Hoàn tất cài đặt spaCy và model ngôn ngữ!


In [2]:
import re
import pandas as pd
from collections import Counter

# Đọc dữ liệu
file_path = "data.xlsx"
df_raw = pd.read_excel(file_path)

# Lấy tên email từ cột "Email"
def get_email_names(df, email_col="Email"):
    email_names = set()
    if email_col in df.columns:
        for e in df[email_col].dropna():
            name = str(e).split("@")[0]
            email_names.add(name.upper())
    return email_names

email_names = get_email_names(df_raw)

# Từ vựng phổ thông loại bỏ
stop_words = {"CONFIRMED", "ORDER", "CODE", "ID"}

# Regex phát hiện URL
url_pattern = re.compile(r"https?://\S+|www\.\S+")

# Regex phát hiện Order ID dạng số ngắn
order_id_number_pattern = re.compile(r"\bOrder\s*ID\s*[:#-]?\s*(\d{5,12})\b", re.IGNORECASE)

# Hàm tách ứng viên mã
def extract_candidates(text):
    # Bỏ URL
    text_no_url = url_pattern.sub(" ", text)

    # Lấy Order ID số ngắn trước
    numeric_ids = [m.group(1) for m in order_id_number_pattern.finditer(text_no_url)]

    # Lấy các mã chữ-số chung, độ dài 5–20
    tokens = re.findall(r"\b[\w-]{5,20}\b", text_no_url)
    candidates = numeric_ids[:]  # copy các ID số trước

    for t in tokens:
        t_up = t.upper()
        if re.search(r"[A-Z]", t) and re.search(r"\d", t):
            if t_up not in email_names and t_up not in stop_words:
                candidates.append(t_up)
    return candidates

# Thu thập tất cả mã từ dữ liệu
all_candidates = []
for body in df_raw["Body"].dropna():
    all_candidates.extend(extract_candidates(body))

# Đếm tần suất, chỉ giữ mã xuất hiện >= 1 lần
counter = Counter(all_candidates)
top_candidates = [c for c, freq in counter.items() if freq >= 1]

# Tạo regex từ các ứng viên
if top_candidates:
    learned_patterns = [fr"\b{re.escape(c)}\b" for c in top_candidates]
else:
    learned_patterns = []

# Hàm trích xuất Order ID chính xác
def extract_order_ids_auto(text: str) -> list:
    text_no_url = url_pattern.sub(" ", text)
    found = set()

    # Lấy ID số ngắn
    for m in order_id_number_pattern.finditer(text_no_url):
        found.add(m.group(1))

    # Lấy các mã học được
    for pat in learned_patterns:
        for m in re.finditer(pat, text_no_url, flags=re.IGNORECASE):
            found.add(m.group(0).upper())
    return sorted(found)

# Chạy trích xuất
order_id = []
for idx, row in df_raw.iterrows():
    email_body = str(row["Body"]).strip()
    if not email_body:
        continue
    order_ids = extract_order_ids_auto(email_body)
    order_id.append({"Order IDs": order_ids})

# In kết quả dạng bảng
if order_id:
    df_results = pd.DataFrame(order_id)
    print(df_results.to_markdown(index=False))

| Order IDs            |
|:---------------------|
| ['GREETINGNET82730'] |
| ['402752216']        |
| ['GREETINGNET82730'] |
| ['BLT9U']            |
| ['GWJSL021983']      |


In [3]:
import re
from collections import Counter
from transformers import pipeline
import spacy
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS as SPACY_STOPWORDS

# Đọc dữ liệu từ file Excel
file_path = "data.xlsx"
df_raw = pd.read_excel(file_path)

# Load mô hình BERT và spaCy
bert_ner = pipeline("ner", model=r"D:\bert-base-NER", aggregation_strategy="simple")
nlp_spacy = spacy.load("en_core_web_sm")

# Tạo danh sách stopwords tùy chỉnh (10 từ khóa liên quan email order confirmation)
custom_stopwords = {
    "order",      # đơn hàng
    "confirmed",  # xác nhận
    "dear",       # mở đầu thư
    "thank",      # lời cảm ơn
    "regards",    # lời kết thư
    "shipment",   # vận chuyển
    "tracking",   # theo dõi đơn hàng
    "invoice",    # hóa đơn
    "customer"
}
all_stopwords = SPACY_STOPWORDS.union(custom_stopwords)

# Hàm học pattern ngày phổ biến 
def learn_patterns(emails):
    date_patterns = []
    for email in emails:
        date_match = re.findall(r"(\b(?:\w+\s+\d{1,2},\s+\d{4}|\d{4}-\d{2}-\d{2})\b)", email)
        date_patterns.extend(date_match)
    common_date = Counter(date_patterns).most_common(1)
    return common_date[0][0] if common_date else None

# Hàm chọn thực thể ít phổ biến nhất, loại bỏ stopwords
def select_most_distinct_entity(entities, text_corpus):
    if not entities:
        return None

    scored_entities = []
    for ent in entities:
        ent_lower = ent.lower().strip()
        words = re.findall(r'\w+', ent_lower)

        # Bỏ nếu entity ngắn quá (ví dụ <= 1 từ)
        if len(words) <= 1:
            continue

        # Bỏ nếu entity chứa bất kỳ từ nào trong custom stopwords (từ khóa email/order)
        if any(word in custom_stopwords for word in words):
            continue

        # Bỏ nếu toàn bộ từ là stopwords (như trước, để chắc chắn)
        if all(word in all_stopwords for word in words):
            continue

        score = len(words) * 10
        frequency = text_corpus.count(ent_lower)
        score -= frequency
        scored_entities.append((ent, score))

    if not scored_entities:
        return None

    scored_entities.sort(key=lambda x: x[1], reverse=True)
    return scored_entities[0][0]


# Hàm trích xuất Product và Date từ 1 email
def extract_info(email_text, text_corpus):
    if not isinstance(email_text, str):
        return {"Product": None, "Date": None}

    # 1. Lấy Date
    date_match = re.search(r"(\b(?:\w+\s+\d{1,2},\s+\d{4}|\d{4}-\d{2}-\d{2})\b)", email_text)
    date = date_match.group(1) if date_match else None

    # 2. NER với BERT và spaCy
    bert_entities = bert_ner(email_text)
    spacy_doc = nlp_spacy(email_text)

    extracted_entities = []

    # Thêm thực thể từ BERT
    for ent in bert_entities:
        if ent["entity_group"] in ["ORG", "PRODUCT", "LOC"]:
            extracted_entities.append(ent["word"])

    # Thêm thực thể từ spaCy
    for ent in spacy_doc.ents:
        if ent.label_ in ["ORG", "PRODUCT", "LOC", "WORK_OF_ART"]:
            extracted_entities.append(ent.text)

    # Chọn cụm từ tốt nhất
    best_product = select_most_distinct_entity(extracted_entities, text_corpus)

    return {"Product": best_product, "Date": date}

# Chuẩn bị dữ liệu: loại bỏ NaN và chuỗi rỗng
emails = df_raw["Body"].dropna()
emails = [str(e).strip() for e in emails if str(e).strip() != ""]

corpus_text = " ".join(emails).lower()
common_date = learn_patterns(emails)

# Trích xuất thông tin
results = [extract_info(email, corpus_text) for email in emails]

# Tạo DataFrame kết quả
df_results = pd.DataFrame(results)

# In kết quả
print(df_results.to_markdown(index=False))


c:\Users\asus\venv\llamaenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\asus\venv\llamaenv\Lib\site-packages\transformers\utils\hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Some weights of the model checkpoint at D:\bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you ex

| Product                       | Date        |
|:------------------------------|:------------|
| Africa Video                  | thg 1, 2025 |
| The Library of Congress - Fri | thg 5, 2025 |
| African Fitness Team × 1      | thg 1, 2025 |
| VÙNG ĐẤT KỲ BÍ                | thg 7, 2025 |
| Global Clearance Warehouse    | thg 7, 2025 |


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
# --- Load mô hình flan-t5-base cho fallback và trả lời ---
gen_model_id = r"D:\flan-t5-base"
gen_tokenizer = AutoTokenizer.from_pretrained(gen_model_id)
gen_model = AutoModelForSeq2SeqLM.from_pretrained(gen_model_id, device_map="cpu")
gen_pipe = pipeline("text2text-generation", model=gen_model, tokenizer=gen_tokenizer)

# --- Hàm tổng hợp trích xuất toàn diện dựa trên kết quả trên ---
def extract_info_combined(email_text: str, text_corpus: str, order_ids_list: list) -> dict:
    product_date_info = extract_info(email_text, text_corpus)
    order_ids = order_ids_list if order_ids_list else []
    product = product_date_info.get("Product")
    date = product_date_info.get("Date")
    if not product or product.strip() == "":
        prompt = f"Extract product names from this email:\n{email_text}\nOnly return product names."
        out = gen_pipe(prompt, max_new_tokens=64, temperature=0.0, num_beams=4, early_stopping=True)
        product = out[0]["generated_text"].strip()
    return {
        "order_ids": list(set(order_ids)),
        "products": [product] if product else [],
        "dates": [date] if date else []
    }

# --- Hàm trả lời câu hỏi dựa trên info ---
def answer_question_with_info(question: str, info: dict) -> str:
    import re
    if re.search(r"order|id", question, flags=re.IGNORECASE):
        if info["order_ids"]:
            return "The order IDs are:\n" + "\n".join(info["order_ids"])
        else:
            return "No order IDs found."
    elif re.search(r"product|item", question, flags=re.IGNORECASE):
        if info["products"]:
            return "The products purchased are:\n" + "\n".join(info["products"])
        else:
            return "No products found."
    elif re.search(r"date|time", question, flags=re.IGNORECASE):
        if info["dates"]:
            return "The date mentioned is:\n" + "\n".join(info["dates"])
        else:
            return "No dates found."
    else:
        context = f"Orders: {info['order_ids']}\nProducts: {info['products']}\nDates: {info['dates']}"
        prompt = f"Based on this information:\n{context}\nAnswer this question: {question}"
        out = gen_pipe(prompt, max_new_tokens=128, temperature=0.0, num_beams=4, early_stopping=True)
        return out[0]["generated_text"]

Device set to use cpu


In [9]:
# --- Danh sách câu hỏi ---
questions = {
    "1": "What product was purchased?",
    "2": "What is the order ID?",
    "3": "When was the email sent?",
    "4": "In tất cả câu trả lời"
}

# Hiển thị menu chọn
print("Chọn câu hỏi bạn muốn trả lời:")
for key, q in questions.items():
    print(f"{key}. {q}")

choice = input("Nhập số tương ứng với câu hỏi: ").strip()

# --- Xử lý từng email ---
for i, (idx, email_body) in enumerate(df_raw["Body"].dropna().items(), start=1):
    print(f"\n--- Email số {i} (index gốc: {idx}) ---")
    
    order_ids_from_email = extract_order_ids_auto(email_body)
    info = extract_info_combined(email_body, corpus_text, order_ids_from_email)

    print("Extracted info:")
    print(info)

    # Trả lời theo lựa chọn
    if choice in ["1", "2", "3"]:
        question_text = questions[choice]
        print(f"\nQ: {question_text}")
        print(answer_question_with_info(question_text, info))
    elif choice == "4":
        for q_key in ["1", "2", "3"]:
            question_text = questions[q_key]
            print(f"\nQ: {question_text}")
            print(answer_question_with_info(question_text, info))
    else:
        print("Lựa chọn không hợp lệ.")
        break


Chọn câu hỏi bạn muốn trả lời:
1. What product was purchased?
2. What is the order ID?
3. When was the email sent?
4. In tất cả câu trả lời

--- Email số 1 (index gốc: 0) ---


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Extracted info:
{'order_ids': ['GREETINGNET82730'], 'products': ['Africa Video'], 'dates': ['thg 1, 2025']}

Q: What product was purchased?
The products purchased are:
Africa Video

Q: What is the order ID?
The order IDs are:
GREETINGNET82730

Q: When was the email sent?
thg 1, 2025

--- Email số 2 (index gốc: 1) ---


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Extracted info:
{'order_ids': ['402752216'], 'products': ['The Library of Congress - Fri'], 'dates': ['thg 5, 2025']}

Q: What product was purchased?
The products purchased are:
The Library of Congress - Fri

Q: What is the order ID?
The order IDs are:
402752216

Q: When was the email sent?
thg 5, 2025

--- Email số 3 (index gốc: 2) ---


...

Extracted info:
{'order_ids': ['GREETINGNET82730'], 'products': ['African Fitness Team × 1'], 'dates': ['thg 1, 2025']}

Q: What product was purchased?
The products purchased are:
African Fitness Team × 1

Q: What is the order ID?
The order IDs are:
GREETINGNET82730

Q: When was the email sent?
thg 1, 2025

--- Email số 4 (index gốc: 3) ---


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Extracted info:
{'order_ids': ['BLT9U'], 'products': ['VÙNG ĐẤT KỲ BÍ'], 'dates': ['thg 7, 2025']}

Q: What product was purchased?
The products purchased are:
VÙNG ĐẤT KỲ BÍ

Q: What is the order ID?
The order IDs are:
BLT9U

Q: When was the email sent?
thg 7, 2025

--- Email số 5 (index gốc: 4) ---


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Extracted info:
{'order_ids': ['GWJSL021983'], 'products': ['Global Clearance Warehouse'], 'dates': ['thg 7, 2025']}

Q: What product was purchased?
The products purchased are:
Global Clearance Warehouse

Q: What is the order ID?
The order IDs are:
GWJSL021983

Q: When was the email sent?
thg 7, 2025
